In [1]:
import os
os.chdir('C:\\Users\\prudi\\Desktop\\Data Sets\\amazon-fine-food-reviews')
#os.chdir('C:\\Users\\Administrator\\Desktop\\Data\\AmazonFineFood\\database.sqlite')
import pandas as pd
import numpy as np
import nltk
import spacy
import sqlite3
import spacy


In [2]:
con = sqlite3.connect('database.sqlite') 
filtered_data = pd.read_sql_query(""" SELECT * FROM Reviews WHERE Score != 3 LIMIT 10""", con)
filtered_data.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [ ]:
## Pre procession the text column

In [8]:
from bs4 import BeautifulSoup

In [67]:
import re
def preprocessing(dataset,column):
    for i in range(len(dataset[column])):
        ## Removing the URL's
        str=re.sub(r'http\S+',' ',dataset[column][i]).strip()
        ## Removing the HTML Tags
        soup=BeautifulSoup(str,'lxml')
        str=soup.get_text()
        # specific
        str = re.sub(r"won't", "will not", str)
        str = re.sub(r"can\'t", "can not", str)
        # general
        str = re.sub(r"n\'t", " not", str)
        str = re.sub(r"\'re", " are", str)
        str = re.sub(r"\'s", " is", str)
        str = re.sub(r"\'d", " would", str)
        str = re.sub(r"\'ll", " will", str)
        str = re.sub(r"\'t", " not", str)
        str = re.sub(r"\'ve", " have", str)
        str = re.sub(r"\'m", " am", str)
        ## Handle digits and text
        str=re.sub(r'\S*\d\S*',' ',str).strip()
        str=re.sub(r'[^A-Za-z0-9]',' ',str).strip()
        dataset[column][i]=str
    return dataset
review_preporcessed=preprocessing(filtered_data,'Text')
review_preporcessed['Text'][0]

C:\Users\prudi\Anaconda3\lib\site-packages\ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


'haveboughtseveraloftheVitalitycanneddogfoodproductsandhavefoundthemalltobeofgoodqualityTheproductlooksmorelikeastewthanaprocessedmeatanditsmellsbetterMyLabradorisfinickyandsheappreciatesthisproductbetterthanmost'

'haveboughtseveraloftheVitalitycanneddogfoodproductsandhavefoundthemalltobeofgoodqualityTheproductlooksmorelikeastewthanaprocessedmeatanditsmellsbetterMyLabradorisfinickyandsheappreciatesthisproductbetterthanmost'

In [43]:
import en_core_web_sm
nlp = en_core_web_sm.load()

In [45]:
for i in range(len(review_preporcessed['Text'])):
    doc=nlp(review_preporcessed['Text'][i])
    filter_list=[]
    for token in doc:
        if token.pos_!='NOUN':
            filter_list.append(token)   
    print(filter_list)
    review_preporcessed['Text'][i]=' '.join(filter_list) 

[have, bought, several, of, the, Vitality, canned, and, have, found, them, all, to, be, of, good, ., The, looks, more, like, a, than, a, processed, and, it, smells, better, ., My, Labrador, is, finicky, and, she, appreciates, this, better, than,  , most, .]


TypeError: sequence item 0: expected str instance, spacy.tokens.token.Token found

In [54]:
a="ave bought several $ of the Vitality canned dog food products and have found them all to be of good quality. The product looks more like a stew than a processed meat and it smells better. My Labrador is finicky and she appreciates this product better than  most."
re.sub(r'[^A-Za-z0-9]',' ',a)


'ave bought several   of the Vitality canned dog food products and have found them all to be of good quality  The product looks more like a stew than a processed meat and it smells better  My Labrador is finicky and she appreciates this product better than  most '

In [20]:
a=[I, have, bought, several, of, the, Vitality, canned, and, have, found, them, all, to, be, of, good, ., The, looks, more, like, a, than, a, processed, and, it, smells, better, My, Labrador, is, finicky, and, she, appreciates, this, better, than,  , most]

SyntaxError: invalid syntax (<ipython-input-20-2f0f2aa8257f>, line 1)

In [1]:
from spacy.pipeline import SentenceSegmenter